In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
#! pip install cudf-cu11 --extra-index-url=https://pypi.nvidia.com -q
#%load_ext cudf.pandas
import pandas as pd

In [ ]:
import os
from glob import glob

#import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from contextlib import contextmanager
from time import time
import matplotlib.pyplot as plt
import seaborn as sns
import random
import shutil

%matplotlib inline


# ref: Kaggleコード遺産 https://qiita.com/kaggle_grandmaster-arai-san/items/d59b2fb7142ec7e270a5
class Timer:
    def __init__(self, logger=None, format_str="{:.3f}[s]", prefix=None, suffix=None, sep=" "):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)


def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)

# 再現性確保!
SEED = 420
seed_everything(SEED)

In [ ]:
import requests
from tqdm import tqdm
from pathlib import Path


COMPETITION_NAME = "atma16"
EXP_NAME = requests.get("http://172.28.0.12:9000/api/sessions").json()[0]["name"][:-6]

HOME = Path(f"/content/drive/MyDrive/{COMPETITION_NAME}")
INPUT_DIR = Path(f"/content/drive/MyDrive/{COMPETITION_NAME}/inputs")
OUTPUT_DIR = Path(f"/content/drive/MyDrive/{COMPETITION_NAME}/outputs/{EXP_NAME}")

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# 学習用のログデータと正解ラベル
train_log_df = pd.read_csv(INPUT_DIR / "train_log.csv")
train_label_df = pd.read_csv(INPUT_DIR / "train_label.csv")

# 宿のデータ
yado_df = pd.read_csv(INPUT_DIR / "yado.csv")

# テスト期間のログデータ
test_log_df = pd.read_csv(INPUT_DIR / "test_log.csv")

sample_submission_df = pd.read_csv(INPUT_DIR / "sample_submission.csv")

# すべてのログデータはあとあと参照をするので先に作っておきます.
whole_log_df = pd.concat([train_log_df, test_log_df], ignore_index=True)

In [ ]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k for a single actual value.

    Parameters:
    actual : int
        The actual value that is to be predicted
    predicted : list
        A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns:
    float
        The average precision at k
    """
    if actual in predicted[:k]:
        return 1.0 / (predicted[:k].index(actual) + 1)
    return 0.0

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k for lists of actual values and predicted values.

    Parameters:
    actual : list
        A list of actual values that are to be predicted
    predicted : list
        A list of lists of predicted elements (order does matter in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns:
    float
        The mean average precision at k
    """
    return sum(apk(a, p, k) for a, p in zip(actual, predicted)) / len(actual)

In [ ]:
def create_top_10_yad_predict(predict_df):
    # セッションごとに予測確率の高い順に yad_no の配列を作成
    _agg = predict_df.sort_values("predict", ascending=False).groupby("session_id")["yad_no"].apply(list)

    out_df = pd.DataFrame(index=_agg.index, data=_agg.values.tolist()).iloc[:, :10]

    return out_df


#create_top_10_yad_predict(best1_raw_pred)

In [ ]:
best1_raw_oof = pd.read_csv('/content/drive/MyDrive/atma16/outputs/exp058_seed890_bagging_frac/exp058_seed890_bagging_frac_oof.csv').rename(columns={'score': 'predict'})
best2_raw_oof = pd.read_csv('/content/drive/MyDrive/atma16/outputs/exp054/exp054_oof.csv').rename(columns={'score': 'predict'})

In [ ]:
oofs = pd.concat([
    best1_raw_oof.set_index(['session_id', 'yad_no'])["predict"],
    best2_raw_oof.set_index(['session_id', 'yad_no'])["predict"]
], axis=1)

,session_id,yad_no,predict
0,000007603d533d30453cc45d0f3d119f,997,0.035102
1,000007603d533d30453cc45d0f3d119f,1168,0.018884
2,000007603d533d30453cc45d0f3d119f,1648,0.029996
3,000007603d533d30453cc45d0f3d119f,2134,0.006155
4,000007603d533d30453cc45d0f3d119f,2445,0.003778
...,...,...,...
10370090,ffe18c66d83d4f7e59649c04292e5cc6,8518,0.006802
10370091,ffe18c66d83d4f7e59649c04292e5cc6,10095,0.007653
10370092,ffe18c66d83d4f7e59649c04292e5cc6,11305,0.008036
10370093,ffe1cfe8f1c0ccb19fe804c8f5d7c43e,5384,0.000420


In [ ]:
oof_label_df = create_top_10_yad_predict(predict_df=oofs.max(axis=1).rename('predict').reset_index())

oof_label_df

,0,1,2,3,4,5,6,7,8,9
session_id,,,,,,,,,,
000007603d533d30453cc45d0f3d119f,2808,11882,5289,4101,997,12846,5821,7157,1648,12837
0000ca043ed437a1472c9d1d154eb49b,8253,11104,4488,2570,1586,8747,6602,13717,825,3725
0000d4835cf113316fe447e2f80ba1c8,9039,5238,918,13642,7509,9854,6676,2957,7796,11031
0000fcda1ae1b2f431e55a7075d1f500,626,11715,13549,1169,1341,755,7812,121,109,12785
000104bdffaaad1a1e0a9ebacf585f33,96,3894,7749,11380,902,5490,12491,1908,12338,10905
...,...,...,...,...,...,...,...,...,...,...
ffff2262d38abdeb247ebd591835dcc9,13210,2876,3725,13717,1586,8992,8677,13079,3564,5179
ffff2360540745117193ecadcdc06538,4767,5299,3940,8703,2900,8465,399,513,963,6654
ffff7fb4617164b2604aaf51c40bf82d,12240,4040,7308,7820,4398,7057,2087,3566,9558,11850


In [ ]:
# いま作成した session_id と同じ並びで train_label を並び替え
train_label = train_label_df.set_index("session_id").loc[oof_label_df.index]["yad_no"].values

In [ ]:
# MAPK (k=10) として計算
mapk(actual=train_label, predicted=oof_label_df.values.tolist(), k=10)

0.3950378695284145

# pred

In [ ]:
best1_raw_pred = pd.read_csv('/content/drive/MyDrive/atma16/outputs/exp058_seed890_bagging_frac/exp058_seed890_bagging_frac_pred_raw.csv').rename(columns={'score': 'predict'})
best2_raw_pred = pd.read_csv('/content/drive/MyDrive/atma16/outputs/exp054/exp054_pred_raw.csv').rename(columns={'score': 'predict'})

In [ ]:
preds = pd.concat([
    best1_raw_pred.set_index(['session_id', 'yad_no'])["predict"],
    best2_raw_pred.set_index(['session_id', 'yad_no'])["predict"]
], axis=1)

In [ ]:
pred_label_df = create_top_10_yad_predict(predict_df=preds.mean(axis=1).rename('predict').reset_index())

In [ ]:
pred_label_df.iloc[:, :10].fillna(0).astype(int).add_prefix("predict_").to_csv(os.path.join(OUTPUT_DIR, f"{EXP_NAME}_submission.csv"), index=False)

In [ ]:
pred_label_df.iloc[:, :10].fillna(0).astype(int).add_prefix("predict_").isnull().sum()

predict_0    0
predict_1    0
predict_2    0
predict_3    0
predict_4    0
predict_5    0
predict_6    0
predict_7    0
predict_8    0
predict_9    0
dtype: int64